## <center>Feature Extraction from Text</center>

This notebook is divided into two sections:
* First, we'll find out what what is necessary to build an NLP system that can turn a body of text into a numerical array of *features* by manually calcuating frequencies and building out TF-IDF.
* Next we'll show how to perform these steps using scikit-learn tools.

### Part One: Core Concepts on Feature Extraction


In this section we'll use basic Python to build a rudimentary NLP system. We'll build a *corpus of documents* (two small text files), create a *vocabulary* from all the words in both documents, and then demonstrate a *Bag of Words* technique to extract features from each document.

For simplicity we won't use any punctuation in the text files One.txt and Two.txt. Let's quickly open them and read them. Keep in mind, you should avoid opening and reading entire files if they are very large, as Python could just display everything depending on how you open the file.


In [1]:
import numpy as np
import pandas as pd

In [2]:
text1 = 'C:/Users/Lenovo/Desktop/Python/Machine Learning/Text data/Data Sets/One.txt'
text2 = 'C:/Users/Lenovo/Desktop/Python/Machine Learning/Text data/Data Sets/Two.txt'

In [3]:
with open(text1) as mytext:
    print(mytext.read())

This is a story about dogs
our canine pets
Dogs are furry animals



In [4]:
with open(text2) as mytext:
    print(mytext.read())

This story is about surfing
Catching waves is fun
Surfing is a popular water sport



#### Building a vocabulary (Creating a "Bag of Words")

Let's create dictionaries that correspond to unique mappings of the words in the documents. We can begin to think of this as mapping out all the possible words available for all (both) documents.

In [5]:
with open(text1) as f:
    words_one = f.read().lower().split()
words_one

['this',
 'is',
 'a',
 'story',
 'about',
 'dogs',
 'our',
 'canine',
 'pets',
 'dogs',
 'are',
 'furry',
 'animals']

In [6]:
len(words_one)

13

In [7]:
uni_words_one = set(words_one)
uni_words_one

{'a',
 'about',
 'animals',
 'are',
 'canine',
 'dogs',
 'furry',
 'is',
 'our',
 'pets',
 'story',
 'this'}

**Repeat for Two.txt**

In [8]:
with open(text2) as f:
    words_two = f.read().lower().split()
    uni_words_two = set(words_two)

uni_words_two

{'a',
 'about',
 'catching',
 'fun',
 'is',
 'popular',
 'sport',
 'story',
 'surfing',
 'this',
 'water',
 'waves'}

**Get all unique words across all documents**

In [9]:
all_uni_words = set()
all_uni_words.update(uni_words_one)
all_uni_words.update(uni_words_two)

all_uni_words

{'a',
 'about',
 'animals',
 'are',
 'canine',
 'catching',
 'dogs',
 'fun',
 'furry',
 'is',
 'our',
 'pets',
 'popular',
 'sport',
 'story',
 'surfing',
 'this',
 'water',
 'waves'}

**Creating an index for each word**

In [10]:
full_vocab = dict()
i = 0

for word in all_uni_words:
    full_vocab[word] = i
    i = i+1

full_vocab

{'canine': 0,
 'animals': 1,
 'surfing': 2,
 'sport': 3,
 'popular': 4,
 'this': 5,
 'our': 6,
 'are': 7,
 'furry': 8,
 'waves': 9,
 'about': 10,
 'story': 11,
 'pets': 12,
 'water': 13,
 'fun': 14,
 'dogs': 15,
 'a': 16,
 'catching': 17,
 'is': 18}

#### Bag of Words to Frequency Counts

Now that we've encapsulated our "entire language" in a dictionary, let's perform *feature extraction* on each of our original documents:

**Empty counts per doc**

In [11]:
# Create an empty vector with space for each word in the vocabulary:
one_freq = np.zeros(len(full_vocab), )
two_freq = np.zeros(len(full_vocab), )
all_words = []

In [12]:
for word in full_vocab.keys():
    all_words.append(word)    

**Add in counts per word per doc:**

In [13]:
# map the frequencies of each word in 1.txt to our vector:
with open(text1) as f:
    one_text = f.read().lower().split()
    
    for word in one_text:
        word_ind = full_vocab[word]
        one_freq[word_ind]+=1
one_freq

array([1., 1., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 2., 1.,
       0., 1.])

In [14]:
# Do the same for the second document:
with open(text2) as f:
    two_text = f.read().lower().split()
    
    for word in two_text:
        word_ind = full_vocab[word]
        two_freq[word_ind]+=1

two_freq

array([0., 0., 2., 1., 1., 1., 0., 0., 0., 1., 1., 1., 0., 1., 1., 0., 1.,
       1., 3.])

In [15]:
pd.DataFrame(data=[one_freq,two_freq],columns=all_words)

,canine,animals,surfing,sport,popular,this,our,are,furry,waves,about,story,pets,water,fun,dogs,a,catching,is
0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,2.0,1.0,0.0,1.0
1,0.0,0.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,3.0


By comparing the vectors we see that some words are common to both, some appear only in `One.txt`, others only in `Two.txt`. Extending this logic to tens of thousands of documents, we would see the vocabulary dictionary grow to hundreds of thousands of words. Vectors would contain mostly zero values, making them **sparse matrices**.




### Concepts to Consider:

#### Bag of Words and Tf-idf
In the above examples, each vector can be considered a *bag of words*. By itself these may not be helpful until we consider *term frequencies*, or how often individual words appear in documents. A simple way to calculate term frequencies is to divide the number of occurrences of a word by the total number of words in the document. In this way, the number of times a word appears in large documents can be compared to that of smaller documents.

However, it may be hard to differentiate documents based on term frequency if a word shows up in a majority of documents. To handle this we also consider *inverse document frequency*, which is the total number of documents divided by the number of documents that contain the word. In practice we convert this value to a logarithmic scale, as described [here](https://en.wikipedia.org/wiki/Tf%E2%80%93idf#Inverse_document_frequency).

Together these terms become [**tf-idf**](https://en.wikipedia.org/wiki/Tf%E2%80%93idf).

#### Stop Words and Word Stems
Some words like "the" and "and" appear so frequently, and in so many documents, that we needn't bother counting them. Also, it may make sense to only record the root of a word, say `cat` in place of both `cat` and `cats`. This will shrink our vocab array and improve performance.

#### Tokenization and Tagging
When we created our vectors the first thing we did was split the incoming text on whitespace with `.split()`. This was a crude form of *tokenization* - that is, dividing a document into individual words. In this simple example we didn't worry about punctuation or different parts of speech. In the real world we rely on some fairly sophisticated *morphology* to parse text appropriately.

Once the text is divided, we can go back and *tag* our tokens with information about parts of speech, grammatical dependencies, etc. This adds more dimensions to our data and enables a deeper understanding of the context of specific documents. For this reason, vectors become ***high dimensional sparse matrices***.

### Part Two:  Feature Extraction with Scikit-Learn

Let's explore the more realistic process of using sklearn to complete the tasks mentioned above!

In [16]:
text = ['This is a line this first line',
           "This is another line",
       "Completely different line"]

#### CountVectorizer

In [17]:
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer,CountVectorizer

In [18]:
countvec = CountVectorizer()
countvec.fit_transform(text) # Stores data in a sparse matrix to avoid using a lot of memory

<3x7 sparse matrix of type '<class 'numpy.int64'>'
	with 11 stored elements in Compressed Sparse Row format>

In [19]:
sparse_mat = countvec.fit_transform(text)

In [20]:
countvec.vocabulary_

{'this': 6,
 'is': 4,
 'line': 5,
 'first': 3,
 'another': 0,
 'completely': 1,
 'different': 2}

In [21]:
sparse_mat.todense() # In the index position of each word, it shows its value count

matrix([[0, 0, 0, 1, 1, 2, 2],
        [1, 0, 0, 0, 1, 1, 1],
        [0, 1, 1, 0, 0, 1, 0]], dtype=int64)

In [22]:
countvec = CountVectorizer(stop_words='english')

In [23]:
countvec.fit_transform(text).todense()

matrix([[0, 0, 2],
        [0, 0, 1],
        [1, 1, 1]], dtype=int64)

In [24]:
countvec.vocabulary_

{'line': 2, 'completely': 0, 'different': 1}

#### TfidfTransformer

TfidfVectorizer is used on sentences, while TfidfTransformer is used on an existing count matrix, such as one returned by CountVectorizer

In [25]:
tfidf_transformer = TfidfTransformer()
cv = CountVectorizer()

In [26]:
counts = cv.fit_transform(text)
counts

<3x7 sparse matrix of type '<class 'numpy.int64'>'
	with 11 stored elements in Compressed Sparse Row format>

In [27]:
tfidf = tfidf_transformer.fit_transform(counts)

In [28]:
tfidf.todense()

matrix([[0.        , 0.        , 0.        , 0.43489333, 0.33074758,
         0.51371001, 0.66149517],
        [0.63174505, 0.        , 0.        , 0.        , 0.4804584 ,
         0.37311881, 0.4804584 ],
        [0.        , 0.65249088, 0.65249088, 0.        , 0.        ,
         0.38537163, 0.        ]])

In [29]:
from sklearn.pipeline import Pipeline

In [30]:
pipe = Pipeline([('cv',CountVectorizer()),('tfidf',TfidfTransformer())])

results = pipe.fit_transform(text)
results.todense()

matrix([[0.        , 0.        , 0.        , 0.43489333, 0.33074758,
         0.51371001, 0.66149517],
        [0.63174505, 0.        , 0.        , 0.        , 0.4804584 ,
         0.37311881, 0.4804584 ],
        [0.        , 0.65249088, 0.65249088, 0.        , 0.        ,
         0.38537163, 0.        ]])

#### TfIdfVectorizer

Does both above in a single step!

In [31]:
tfidf = TfidfVectorizer()

new = tfidf.fit_transform(text)
new.todense()

matrix([[0.        , 0.        , 0.        , 0.43489333, 0.33074758,
         0.51371001, 0.66149517],
        [0.63174505, 0.        , 0.        , 0.        , 0.4804584 ,
         0.37311881, 0.4804584 ],
        [0.        , 0.65249088, 0.65249088, 0.        , 0.        ,
         0.38537163, 0.        ]])